In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.models import inception_v3, Inception3
from torchvision.models import inception_v3, Inception_V3_Weights

# Resize to fit input of CNN
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
])

# Load data and apply resize
train_dataset = ImageFolder('train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Import Inception model
inception = inception_v3(pretrained = True)

# Freeze the parameters in the pre-trained layers
for param in inception.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to fit the number of classes in your task
num_classes = 2  # Melanoma and melanocytic nevus
inception.fc = nn.Linear(inception.fc.in_features, num_classes)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(inception.parameters(), lr=0.001)


C:\Users\Fran\anaconda3\envs\cuda_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Fran\anaconda3\envs\cuda_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
import torch.nn.functional as F
# Define number of epochs
num_epochs = 10

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inception = inception.to(device)


for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # One-hot encode the target labels
        target = F.one_hot(labels, num_classes=2).float()  # Assuming 2 classes
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs, _ = inception(inputs)  # Use the main output of the Inception model
        loss = criterion(outputs, target)  # Use one-hot encoded target
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        
        # Calculate accuracy
        predicted = torch.argmax(torch.sigmoid(outputs), dim=1)  # Get the index of the max logit as prediction
        total += labels.size(0)
        correct += (predicted == labels).sum().item()  # Compare predicted indices with true labels
    
    epoch_loss = running_loss / len(train_dataset)
    epoch_accuracy = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

print("Training finished.")





Epoch [1/10], Loss: 0.4918, Accuracy: 0.7694
Epoch [2/10], Loss: 0.4222, Accuracy: 0.8005
Epoch [3/10], Loss: 0.4112, Accuracy: 0.8091
Epoch [4/10], Loss: 0.4100, Accuracy: 0.8126
Epoch [5/10], Loss: 0.3973, Accuracy: 0.8129
Epoch [6/10], Loss: 0.3932, Accuracy: 0.8238
Epoch [7/10], Loss: 0.3881, Accuracy: 0.8224
Epoch [8/10], Loss: 0.3998, Accuracy: 0.8129
Epoch [9/10], Loss: 0.3966, Accuracy: 0.8161
Epoch [10/10], Loss: 0.3902, Accuracy: 0.8235
Training finished.


In [3]:
# Assuming similar data transformations for validation set
val_dataset = ImageFolder('val', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)  # No need to shuffle for validation


# Evaluation mode
inception.eval()


# Iterate over validation data
# Variables to track accuracy and loss
val_correct = 0
val_total = 0
val_running_loss = 0.0

# Iterate over validation data
for inputs, labels in val_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    # One-hot encode the target labels
    target = F.one_hot(labels, num_classes=num_classes).float()
    
    # Forward pass
    with torch.no_grad():
        outputs = inception(inputs)  # Use the main output of the Inception model
        loss = criterion(outputs, target)  # Use one-hot encoded target
    
    # Calculate loss
    val_running_loss += loss.item() * inputs.size(0)
    
    # Calculate accuracy
    predicted = (torch.sigmoid(outputs) > 0.5).float()  # Convert probabilities to binary predictions
    val_total += labels.size(0)
    val_correct += (predicted == target).all(dim=1).sum().item()  # Compare with one-hot encoded target

# Calculate validation loss and accuracy
val_loss = val_running_loss / len(val_dataset)
val_accuracy = val_correct / val_total

print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")



Validation Loss: 0.3536, Validation Accuracy: 0.8371
